# Simple LLM app

In [1]:
from langchain_community.llms import Ollama

In [2]:
llm = Ollama(model='llama3')

In [3]:
llm

Ollama(model='llama3')

In [4]:
response = llm.invoke("Do you know about Claude 3?")
print(response)

I think you might be referring to Cloude or Claudie 3, a popular character in the French comic book series "Lucky Luke" created by Morris. In this context, Claude is not a person but rather an old stagecoach that appears in several episodes of the series.

If I'm correct, please tell me more about what you'd like to know about Cloude 3 or Lucky Luke! If not, feel free to provide more information or clarify your question.


## Create a prompt template

In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([("system","You are an Artificial Intelligence News Reporter"),("user","{query}")])

In [6]:
llm_app = prompt | llm
llm_app

ChatPromptTemplate(input_variables=['query'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an Artificial Intelligence News Reporter')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], template='{query}'))])
| Ollama(model='llama3')

In [7]:
response = llm_app.invoke({"query":"Do you know about Claude 3?"})
response

'Breaking news, folks! I\'ve got a scoop for you. Yes, I\'m familiar with Claude 3. For those who may not be in the loop, Claude 3 is an artificial intelligence language model designed by Google to generate human-like text responses.\n\nAccording to recent reports, Claude 3 has been trained on a massive dataset of text from the internet and can respond to user input in a way that\'s both informative and engaging. The AI is said to be capable of understanding natural language processing, allowing it to understand context and follow conversations.\n\nNow, I know what you\'re thinking: "Is this just another chatbot?" Not quite, folks. Claude 3 has been touted as one of the most advanced language models out there, with the potential to revolutionize the way we interact with machines.\n\nI\'ve got sources suggesting that Google is planning to integrate Claude 3 into various applications, from customer service tools to creative writing platforms. Imagine having a conversation with an AI that

## RAG

In [5]:
!pip install beautifulsoup4

  Obtaining dependency information for beautifulsoup4 from https://files.pythonhosted.org/packages/b1/fe/e8c672695b37eecc5cbf43e1d0638d88d66ba3a44c4d321c796f4e59167f/beautifulsoup4-4.12.3-py3-none-any.whl.metadata
  Using cached beautifulsoup4-4.12.3-py3-none-any.whl.metadata (3.8 kB)
Using cached beautifulsoup4-4.12.3-py3-none-any.whl (147 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
nbconvert 7.16.3 requires bleach!=5.0.0, which is not installed.
jupyter-server 2.14.0 requires argon2-cffi>=21.1, which is not installed.


In [8]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.anthropic.com/news/claude-3-family")

docs = loader.load()

In [9]:
urls = [
        "https://www.anthropic.com/news/releasing-claude-instant-1-2",
        "https://www.anthropic.com/news/claude-pro",
        "https://www.anthropic.com/news/claude-2",
        "https://www.anthropic.com/news/claude-2-1",
        "https://www.anthropic.com/news/claude-2-1-prompting",
        "https://www.anthropic.com/news/claude-3-family",
        "https://www.anthropic.com/claude"
       ] 

docs = []
for url in urls:
    loader = WebBaseLoader(url)
    docs.extend(loader.load())
    
len(docs)

7

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings_llm = OllamaEmbeddings(model="llama3") #base_url = 'http://localhost:11434'

In [9]:
embeddings = embeddings_llm.embed_query("How are you?")

embeddings[:5]

[-0.4689432680606842,
 -0.30772146582603455,
 2.4219796657562256,
 0.2532179057598114,
 -0.10468652099370956]

In [10]:
type(embeddings), len(embeddings)


(list, 4096)

In [11]:
embeddings = embeddings_llm.embed_documents([
                                "Claude 3 is latest Conversational AI Model from Anthropic.",
                                "Gemini is latest Conversational AI Model from Google.",
                                "Llama-3 is latest Conversational AI Model from Meta.",
                                "Mixtral is latest Conversational AI Model from Mistral AI.",
                                "GPT-4 is latest Conversational AI Model from OpenAI."
                               ])

len(embeddings), type(embeddings), len(embeddings[0])

(5, list, 4096)

In [11]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

documents = text_splitter.split_documents(docs)

len(documents)

12

In [15]:
!pip install faiss-cpu

  Obtaining dependency information for faiss-cpu from https://files.pythonhosted.org/packages/c3/d3/2465dda67d48c873fcce7dbfa8018a0ae648e150e4d336c84e8ba553c897/faiss_cpu-1.8.0-cp311-cp311-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 32.8 MB/s eta 0:00:00a 0:00:01


In [12]:
from langchain_community.vectorstores import FAISS

vector_index = FAISS.from_documents(documents, embeddings_llm)

vector_index

### There are 3 functions to create index.

from_documents()
from_embeddings()
from_texts()

In [13]:
retriever = vector_index.as_retriever()

relevant_docs = retriever.invoke({"input": "Do you know about Claude 3?"})

len(relevant_docs)

4

In [14]:
relevant_docs


[Document(page_content='while also exhibiting reduced levels of incorrect answers.In addition to producing more trustworthy responses, we will soon enable citations in our Claude 3 models so they can point to precise sentences in reference material to verify their answers.Long context and near-perfect recallThe Claude 3 family of models will initially offer a 200K context window upon launch. However, all three models are capable of accepting inputs exceeding 1 million tokens and we may make this available to select customers who need enhanced processing power.To process long context prompts effectively, models require robust recall capabilities. The \'Needle In A Haystack\' (NIAH) evaluation measures a model\'s ability to accurately recall information from a vast corpus of data. We enhanced the robustness of this benchmark by using one of 30 random needle/question pairs per prompt and testing on a diverse crowdsourced corpus of documents. Claude 3 Opus not only achieved near-perfect re

In [15]:
for doc in relevant_docs:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Claude 2 \ Anthropic, Source: https://www.anthropic.com/news/claude-2
Title : Releasing Claude Instant 1.2 \ Anthropic, Source: https://www.anthropic.com/news/releasing-claude-instant-1-2
Title : Claude \ Anthropic, Source: https://www.anthropic.com/claude


In [16]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document

prompt = ChatPromptTemplate.from_template("""
Answer the following question based on the provided context and your internal knowledge.
Give priority to context and if you are not sure then say you are not aware of topic:

<context>
{context}
</context>

Question: {input}
""")

#document_chain  = prompt | llm 
document_chain = create_stuff_documents_chain(llm, prompt)

response = document_chain.invoke({
    "input": "Do you know about Claude 3?",
    "context": [Document(page_content="Claude 3 is latest Conversational AI Model from Anthropic.")]
})

print(response)

Yes, I am aware of the context provided. According to the context, Claude 3 is the latest Conversational AI model from Anthropic.


In [18]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever, document_chain)

retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x109aa6990>), config={'run_name': 'retrieve_documents'})
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), config={'run_name': 'format_inputs'})
            | ChatPromptTemplate(input_variables=['context', 'input'], messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'input'], template='\nAnswer the following question based on the provided context and your internal knowledge.\nGive priority to context and if you are not sure then say you are not aware of topic:\n\n<context>\n{context}\n</context>\n\nQuestion: {input}\n'))])
            | Ollama(model='llama3')
            | Str

In [19]:
response = retrieval_chain.invoke({"input": "Do you know about Claude 3?"})

type(response)

dict

In [20]:
print(response["answer"])

Claude 3 is a powerful language model developed by Anthropic, a leading AI research organization. It's designed to be highly intelligent and capable of understanding complex tasks, with three variants: Opus, Sonnet, and Haiku.

Opus is the most intelligent model, ideal for complex analysis, longer tasks with multiple steps, and higher-order math and coding tasks.

Sonnet is engineered for high endurance in large-scale AI deployments, offering a balance between intelligence and speed at a lower cost compared to its peers.

Haiku is the fastest and most compact model, perfect for near-instant responsiveness and answering simple queries and requests quickly.

Each variant has its unique strengths and use cases, making Claude 3 an excellent choice for various applications, from coding assistance to data processing and content moderation.

What would you like to know more about Claude 3 or how it can be applied in a specific context?


In [21]:
for doc in response["context"]:
    print(f"Title : {doc.metadata['title']}, Source: {doc.metadata['source']}")

Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
Title : Claude \ Anthropic, Source: https://www.anthropic.com/claude
Title : Claude 2 \ Anthropic, Source: https://www.anthropic.com/news/claude-2
Title : Introducing the next generation of Claude \ Anthropic, Source: https://www.anthropic.com/news/claude-3-family
